# Sprint 5 – Preparação Final para o Dashboard
Chegando na última sprint! Aqui o foco é preparar os dados finais que vão ser usados no dashboard da Zents. Vou juntar tudo o que foi criado até agora em um único conjunto de dados.

In [3]:
# Bibliotecas
import pandas as pd

## 1. Carregando os dados necessários
Vou carregar os dados de pedidos, itens e pagamentos para juntar tudo.

In [4]:
pedidos = pd.read_csv('../data/raw/olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp', 'order_delivered_customer_date'])
itens = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
pagamentos = pd.read_csv('../data/raw/olist_order_payments_dataset.csv')
clientes = pd.read_csv('../data/raw/olist_customers_dataset.csv')
vendedores = pd.read_csv('../data/raw/olist_sellers_dataset.csv')

## 2. Criando colunas úteis para o dashboard

In [5]:
# Tempo de entrega (em dias)
pedidos['tempo_entrega'] = (pedidos['order_delivered_customer_date'] - pedidos['order_purchase_timestamp']).dt.days

In [6]:
# Soma do valor dos itens por pedido
total_itens = itens.groupby('order_id')['price'].sum().reset_index(name='valor_total_itens')

In [7]:
# Soma dos pagamentos por pedido
total_pagamento = pagamentos.groupby('order_id')['payment_value'].sum().reset_index(name='valor_pagamento')

In [14]:
# Juntando tudo em um único DataFrame
df = pedidos[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'tempo_entrega']].copy()
df = df.merge(total_itens, on='order_id', how='left')
df = df.merge(total_pagamento, on='order_id', how='left')
df = df.merge(clientes[['customer_id', 'customer_state']], left_on='order_id', right_on='customer_id', how='left')
df = df.merge(itens[['order_id', 'seller_id']], on='order_id', how='left')

In [15]:
# Criando a coluna ano_mes para filtros temporais
df['ano_mes'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)

## 3. Exportando o dataset final
Esse será o arquivo usado no dashboard.

In [18]:
# Salvando como CSV
df.to_csv('../data/processed/zents_dashboard_dataset_final.csv', index=False)
print('✅ Dataset final exportado com sucesso!')

✅ Dataset final exportado com sucesso!


## 4. Conclusão
Pronto! Agora temos um dataset completo e limpo com:
- Tempo de entrega
- Valores pagos e dos produtos
- Estado do cliente
- Loja que vendeu
- Mês/Ano do pedido

Esse arquivo pode ser usado direto na dashboard do Streamlit.